<img src="images/Project_logos.png" width="500" height="300" align="center">

## Aims

This course will teach you some advanced techniques to investigate and handle correlations in your data.


Prior knowledge of Python, NumPy, Pandas, Iris, and Matplolib are assumed for this course.

## Table of Contents

* [Autocorrelation](#autocorrelation)
* [Examples using NumPy](#examples_numpy)
* [Examples using Pandas](#examples_pandas)
* [Empirical Orthogonal Functions](#eofs)
* [Exercise 1](#exercise1)

## Autocorrelation<a class="anchor" id="autocorrelation"></a>

Environmental data are usually collected as snapshots in time rather than as continuous variables, for example measurements may be taken hourly, daily, monthly, yearly. The closer in time the measurements are taken, the more similar the measurements will be: an hourly datapoint will be more similar to the previous hourly datapoint than a yearly datapoint will be to the previous yearly datapoint.  This means that data can be **autocorrelated** across a given time lag. 

Autocorrelation is usually calculated in terms of different lag times and is useful to identify the dependence, persistence or memory within the data. Temporal correlation can also be termed **serial correlation** or **lagged correlation** and is essentially the correlation between a datapoint and a datapoint from the previous time steps of data collection.

- **Positive Autocorrelation**: an increase in one datapoint value predicts an increase in another datapoint value at a different timestep
- **Negative Autocorrelation**: an increase in one datapoint value predicts a decrease in another datapoint value at a different timestep


## Examples using the NumPy library<a class="anchor" id="examples_numpy"></a>

In [ ]:
import numpy as np
from numpy import genfromtxt
import matplotlib.pyplot as plt

# Load in the data
data = genfromtxt('data/heathrow_weather_station_data.csv', delimiter=',')

# select the maximum temperature data
tmax = data[1:,2]

# calculate the autocorrelation with a time lag of 1 month
lag1_autocorrelation = np.corrcoef(tmax[1:-1], tmax[2:])[0,1]
print(f'Lag 1 month autocorrelation = {lag1_autocorrelation}\n')

# plot the autocovariances for 100 lags
# Note that here you can see the seasonal cycle, and that the relationship
# between the monthly values and the values for previous years diminishes
# as the lag increases.
fig = plt.figure()
plt.xlabel("Lag (months)")
plt.ylabel("Autocorrelation")
plt.acorr(tmax, maxlags = 100)
plt.show()

<font color='red'>**NOTE**</font>:  The statistical autocorrelation is normalized to be between -1 and 1, whereas the signal processing definition is not, and any unstandardised autocorrelations, such as that plotted above, are more correctly termed autocovariances.

In [ ]:
# To plot the autocorrelations using this method, first normalize the data:
mean = sum(tmax) / len(tmax) 
var = sum([(x - mean)**2 for x in tmax]) / len(tmax) 
# Normalized data
tmax = [x - mean for x in tmax]

fig = plt.figure()
plt.xlabel("Lag (months)")
plt.ylabel("Autocorrelation")
plt.acorr(tmax, maxlags = 100)
plt.show()

## Examples using the Pandas library<a class="anchor" id="examples_pandas"></a>

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from pandas.plotting import autocorrelation_plot
from statsmodels.graphics import tsaplots

# Load in the data
df_data = pd.read_csv('data/heathrow_weather_station_data.csv')

# print the monthly data
print(df_data)

# select the maximum temperature data
df_tmax = df_data['Max_temperature']

# calculate the autocorrelation with a time lag of 1 month
lag1_autocorrelation = df_tmax.autocorr()
print(f'\n\nLag 1 month autocorrelation = {lag1_autocorrelation}\n')

# plot the relationship between the monthly maximum temperature
# and the maximum temperature the following month
pd.plotting.lag_plot(df_tmax, lag=1)

# plot the autocorrelations for all lags
fig = plt.figure()
plt.xlabel("Lag (months)")
plt.ylabel("Autocorrelation")
autocorrelation_plot(df_tmax)



In [ ]:
from statsmodels.graphics import tsaplots

# another version of plotting the autocorrelations for 100 lags
# in this plot the shading shows the 95% confidence interval
# meaning that any datapoints within the shaded area have no 
# significant correlation with the most recent datapoint.
fig = tsaplots.plot_acf(df_tmax, lags=100)
plt.show() 

## Empirical Orthogonal Functions<a class="anchor" id="eofs"></a>
Data can also be correlated in space and a common method for examining patterns in data that varies across space and time is to create Empirical Orthogonal Function (EOFs) that decompose the data into a spatial component and a temporal component. A complete description of EOF analysis is beyond the scope of this course and it is recommended that further reading be conducted. Here only the methods for conducting an EOF analysis in Python are demonstrated.

Here we provide an example of using sea surface temperature (SST) data to identify leafing modes of climate variability.

In [ ]:
import iris
import numpy as np
from eofs.iris import Eof
import pandas as pd
import cartopy.crs as ccrs
import matplotlib.pyplot as plt
import iris.plot as iplt

def detrend_cube(cube, order, dim=0):
    # returns the array of regression coeficients, the fitted y values (same size as cube) and the
    # cube of the detrended data
    coord = cube.coords(contains_dimension=dim, dim_coords=True)[0]
    cs = cube.shape
    A2 = cube.data.reshape(cs[dim], -1)
    reg = np.polyfit(coord.points, A2, order)
    yfit = [np.polyval(reg[:, x], coord.points) for x in range(reg.shape[1])]
    yfitarr = np.array(yfit).T.reshape(cs)
    ncube = cube.copy()

    ncube.data = ncube.data-yfitarr
    # retain original mask if available
    if isinstance(cube.data, np.ma.MaskedArray):
        ncube.data = np.ma.MaskedArray(ncube.data, mask=cube.data.mask)
    return reg, yfitarr, ncube


# Load in the SST data for December-March
cube = iris.load_cube('data/OBS-ERA5_5deg_sst_jfm_1979_2018_low_res_5deg.nc')

# Linearly detrend the data
_, _, cube = detrend_cube(cube, 1, dim=0)

solver = Eof(cube, weights='coslat')
eofs = solver.eofsAsCovariance(neofs=4)
eigenvalues = solver.eigenvalues()
pcs = solver.pcs(npcs=4, pcscaling=1)
    
# Plot the spatial pattern of the leading 4 EOFs expressed as covariance.
fig = plt.figure()

for e in [0,1,2,3]:
    ax = fig.add_subplot(2,2,e+1, projection=ccrs.PlateCarree())
    iplt.contourf(eofs[e], cmap=plt.cm.RdBu_r)
    ax.set_title(f'EOF{e+1} ({int((eigenvalues.data[e]/eigenvalues.data.sum())*100)}% of variance explained)', fontsize=10)
    ax.coastlines()
    ax.set_global()
plt.tight_layout()
plt.show


Since the first EOF looks like the El Nino Southern Oscillation (ENSO), we can plot this as a timeseries

In [ ]:
# Load in the SST data for December-March
cube = iris.load_cube('data/OBS-ERA5_5deg_sst_jfm_1979_2018_low_res_5deg.nc')

# Linearly detrend the data
_, _, cube = detrend_cube(cube, 1, dim=0)

# Subset the ENSO region (5S-5N and 170-120W)
subset_cube = cube.intersection(longitude=(-170, -120), latitude=(-5, 5))

# Calculate the EOFs for the ENSO region
solver = Eof(subset_cube, weights='coslat')
eofs = solver.eofsAsCovariance(neofs=1)
eigenvalues = solver.eigenvalues()
pcs = solver.pcs(npcs=1, pcscaling=1)

# Save the timeseries to a Pandas DataFrame
pc_df = pd.DataFrame(pcs.data, columns=['EOF1'])
pc_df['Year'] = cube.coord('season_year').points

# Plot the timeseries
fig = plt.figure()
ax = fig.add_subplot(111)
ax.plot(pc_df['Year'], pc_df[f'EOF1'], color='k', linewidth=0.5)
ax.set_title(f'EOF1 (ENSO)', fontsize=16)
pc_df[f'POS_EOF1'] = 0
pc_df[f'NEG_EOF1'] = 0
pos_threshold = 0.5
neg_threshold = -0.5
pc_df.loc[pc_df[f'EOF1'] >pos_threshold, f'POS_EOF1'] = 1
pc_df.loc[pc_df[f'EOF1'] <neg_threshold, f'NEG_EOF1'] = 1
num_pos = pc_df[f'POS_EOF1'].sum()
num_neg = pc_df[f'NEG_EOF1'].sum()
plt.fill_between(pc_df['Year'], pos_threshold, pc_df[f'EOF1'], color='red',
                 where=pc_df[f'EOF1']>=pos_threshold, interpolate=True)
plt.fill_between(pc_df['Year'], neg_threshold, pc_df[f'EOF1'], color='blue',
                 where=pc_df[f'EOF1']<=neg_threshold, interpolate=True)
plt.axhline(y=0, color='k', linestyle='-', linewidth=0.5)
plt.text(1978, -3.0, f'{num_pos} El Nino events', horizontalalignment='left',
         verticalalignment='center')
plt.text(1978, -3.3, f'{num_neg} La Nina events', horizontalalignment='left',
         verticalalignment='center')
plt.tight_layout()
plt.show

## Exercise 1<a class="anchor" id="exercise_1"></a>

Using any Python methods, calculate and plot a map and the timeseries for the Summer North Atlantic Oscillation (SNAO) defined as the first EOF of June-August mean sea level pressure for region 40N–70N, 90W–30E.

In [ ]:
import iris

cube = iris.load_cube('data/OBS-ERA5_5deg_mslp_jja_1979_2018_low_res_5deg.nc')
    


<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
**Solution**

<font color='red'>**NOTE**</font>: Your methods can include any Python library



In [ ]:
import iris

cube = iris.load_cube('data/OBS-ERA5_5deg_mslp_jja_1979_2018_low_res_5deg.nc')
    
# Subset the SNAO region (40N–70N, 90W–30E)
subset_cube = cube.intersection(longitude=(-90, 30), latitude=(40, 70))
print(subset_cube)

# Calculate the EOFs for the SNAO region
solver = Eof(subset_cube, weights='coslat')
eofs = solver.eofsAsCovariance(neofs=1)
eigenvalues = solver.eigenvalues()
pcs = solver.pcs(npcs=1, pcscaling=1)

# Plot the spatial pattern of the leading EOF expressed as covariance.
fig = plt.figure()

ax = fig.add_subplot(111, projection=ccrs.PlateCarree())
iplt.contourf(eofs[0], cmap=plt.cm.RdBu_r)
ax.set_title(f'EOF1 ({int((eigenvalues.data[e]/eigenvalues.data.sum())*100)}% of variance explained)', fontsize=10)
ax.coastlines()
ax.set_global()
plt.tight_layout()

# Save the timeseries to a Pandas DataFrame
pc_df = pd.DataFrame(pcs.data, columns=['EOF1'])
pc_df['Year'] = subset_cube.coord('season_year').points

# Plot the timeseries
fig = plt.figure()
ax = fig.add_subplot(111)
ax.plot(pc_df['Year'], pc_df[f'EOF1'], color='k', linewidth=0.5)
ax.set_title(f'EOF1 (SNAO)', fontsize=16)
pc_df[f'POS_EOF1'] = 0
pc_df[f'NEG_EOF1'] = 0
pos_threshold = 0.5
neg_threshold = -0.5
pc_df.loc[pc_df[f'EOF1'] >pos_threshold, f'POS_EOF1'] = 1
pc_df.loc[pc_df[f'EOF1'] <neg_threshold, f'NEG_EOF1'] = 1
num_pos = pc_df[f'POS_EOF1'].sum()
num_neg = pc_df[f'NEG_EOF1'].sum()
plt.fill_between(pc_df['Year'], pos_threshold, pc_df[f'EOF1'], color='red',
                 where=pc_df[f'EOF1']>=pos_threshold, interpolate=True)
plt.fill_between(pc_df['Year'], neg_threshold, pc_df[f'EOF1'], color='blue',
                 where=pc_df[f'EOF1']<=neg_threshold, interpolate=True)
plt.axhline(y=0, color='k', linestyle='-', linewidth=0.5)
plt.text(1978, -3.0, f'{num_pos} Positive SNAO events', horizontalalignment='left',
         verticalalignment='center')
plt.text(1978, -3.3, f'{num_neg} Negative SNAO events', horizontalalignment='left',
         verticalalignment='center')
plt.tight_layout()
plt.show